In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import roc_auc_score


### Loading data

In [23]:
# Comp 1
#train_data = pd.read_csv("C:/Users/johne/Downloads/playground-series-s4e7/train.csv")
#test_data = pd.read_csv("C:/Users/johne/Downloads/playground-series-s4e7/test.csv")

# Comp 2
train_data = pd.read_csv("C:/Users/matt/Downloads/train.csv/train.csv")
test_data = pd.read_csv("C:/Users/matt/Downloads/test.csv/test.csv")


# Display the first few rows of each dataframe
print(train_data.head())
print(test_data.head())

   id  Gender  Age  Driving_License  Region_Code  Previously_Insured  \
0   0    Male   21                1         35.0                   0   
1   1    Male   43                1         28.0                   0   
2   2  Female   25                1         14.0                   1   
3   3  Female   35                1          1.0                   0   
4   4  Female   36                1         15.0                   1   

  Vehicle_Age Vehicle_Damage  Annual_Premium  Policy_Sales_Channel  Vintage  \
0    1-2 Year            Yes         65101.0                 124.0      187   
1   > 2 Years            Yes         58911.0                  26.0      288   
2    < 1 Year             No         38043.0                 152.0      254   
3    1-2 Year            Yes          2630.0                 156.0       76   
4    1-2 Year             No         31951.0                 152.0      294   

   Response  
0         0  
1         1  
2         0  
3         0  
4         0  
        

In [24]:

# Split features and target
X = train_data.drop(['Response', 'id'], axis=1)
y = train_data['Response']
test_ids = test_data['id']
X_test = test_data.drop(['id'], axis=1)

# Identify numerical and categorical columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

In [27]:
# Check the data types of Region_Code and Age
print(train_data[['Region_Code', 'Age']].dtypes)
print(test_data[['Region_Code', 'Age']].dtypes)

Region_Code    float64
Age              int64
dtype: object
Region_Code    float64
Age              int64
dtype: object


In [31]:
# Feature Engineering
# Function to process a chunk of data
def process_chunk(df):
    df['Region_Code'] = df['Region_Code'].astype(int).astype(str)
    df['Age'] = df['Age'].astype(str)
    df['Region_Code_Age'] = df['Region_Code'] + '_' + df['Age']
    df = pd.get_dummies(df, columns=['Region_Code_Age'], drop_first=True)
    return df

# Process train_data in chunks
chunk_size = 500  # Adjust chunk size based on your memory capacity
train_chunks = []
for chunk in pd.read_csv('C:/Users/matt/Downloads/train.csv/train.csv', chunksize=chunk_size):
    processed_chunk = process_chunk(chunk)
    train_chunks.append(processed_chunk)

train_data_processed = pd.concat(train_chunks, ignore_index=True)

# Process test_data in chunks
test_chunks = []
for chunk in pd.read_csv('C:/Users/matt/Downloads/test.csv/test.csv', chunksize=chunk_size):
    processed_chunk = process_chunk(chunk)
    test_chunks.append(processed_chunk)

test_data_processed = pd.concat(test_chunks, ignore_index=True)



ParserError: Error tokenizing data. C error: out of memory

### Preprocessing

In [ ]:
# Preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])


### Defining Model

In [ ]:
# Define the model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])


### Training 

In [ ]:

# Split data for validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Train the model
model.fit(X_train, y_train)

c:\Users\matt\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:50:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Annual_Premium', 'Policy_Sales_Channel', 'Vintage'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 st...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

### Testing

In [ ]:
# Predict on validation set and evaluate
y_valid_pred = model.predict_proba(X_valid)[:, 1]
roc_auc = roc_auc_score(y_valid, y_valid_pred)
print(f'Validation ROC AUC: {roc_auc}')

Validation ROC AUC: 0.8782425125130968


In [ ]:
# Predict on the test set
test_predictions = model.predict_proba(X_test)[:, 1]

### Preparing Submission

In [ ]:
# Prepare the submission file
submission = pd.DataFrame({'id': test_ids, 'Response': test_predictions})
submission.to_csv('C:/Users/matt/Downloads/submission3.csv', index=False)
print('Submission file created successfully!')

Submission file created successfully!
